# Conformal prediction for binary classification

Here we show how a binary classifier can be applied to produce one of four classifications:

- *Positive*: The instance reaches the threshold for the positive class, but not the negative class.

- *Negative*: The instance reaches the threshold for the negative class, but not the positive class.

- *Both*: The instance reaches the threshold for both positive and negative classes.

- *None*: The instance does not reach the threshold for either class.

Adjusting $\alpha$ (*coverage*) will adjust the balance of classes. A high $\alpha$ is more likely to classify na instance in both classes, whereas a low $\alpha$ is more likely to fail to classify in either class.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from mapie.classification import MapieClassifier
from mapie.metrics import classification_coverage_score
from mapie.metrics import classification_mean_width_score

## Create data

Example data will be produced using SK-Learn's `make_blobs` method.

In [ ]:
n_classes = 2
# Make train and test data using sklearn's make_classification
X, y = make_classification(n_samples=10000, n_classes=2, n_features=5,
                           n_informative=5, n_redundant=0, n_clusters_per_class=1,
                           class_sep=0.7, random_state=42)
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

## Train model and MAPIE classifier

MAPIE wraps around any model. It fits the model and gets the prediction sets.

The `score` method is the same as used above. This method may be used with k-fold MAPIE fitting which fits a model and then gives us prediction sets across the whole test set by using k-fold calibration/test. 

In [ ]:
# Define classifier
classifier = LogisticRegression(random_state=42)
mapie_score = MapieClassifier(estimator=classifier, method='score', cv=5)
# Fit classifier and get predictions
mapie_score.fit(X_train, y_train)
y_pred, y_set = mapie_score.predict(X_test, alpha=0.05)
# Remove redundant dimension (used if more than one alpha is used)
y_set = np.squeeze(y_set)
# Show first 5 instances
print(y_set[0:5])

## Analyse classifications

Full coverage

In [ ]:
# Full coverage
cov = classification_coverage_score(y_test, y_set)
setsize = classification_mean_width_score(y_set)
print(f'Coverage: {cov:0.2f}')
print(f'Avg. set size: {setsize:.2f}')

Class-wise performance

In [ ]:
def class_wise_performance(y_new, y_set, n_classes):
    df = pd.DataFrame()
    # Loop through the classes
    for i in range(n_classes):
    # Calculate the coverage and set size for the current class
        ynew = y_new[y_new == i]
        yscore = y_set[y_new == i]
        cov = classification_coverage_score(ynew, yscore)
        size = classification_mean_width_score(yscore)
        # Create a new dataframe with the calculated values
        temp_df = pd.DataFrame({
            "class": [i],
            "coverage": [cov],
            "avg. set size": [size]
            }, index = [i])
        # Concatenate the new dataframe with the existing one
        df = pd.concat([df, temp_df])
    df.set_index('class', inplace=True)
    return(df)

In [ ]:
# Get class-wise performance
print('Class wise performance')
print(class_wise_performance(y_test, y_set, n_classes))

Count how many instances are in the following classes:

- *Positive*: The instance reaches the threshold for the positive class, but not the negative class.

- *Negative*: The instance reaches the threshold for the negative class, but not the positive class.

- *Both*: The instance reaches the threshold for both positive and negative classes.

- *None*: The instance does not reach the threshold for either class.

In [ ]:
def count_classification(row):
    if (row[0] == False) & (row[1] == True):
        return 'Positive'    
    elif (row[0] == True) & (row[1] == False):
        return 'Negative'
    elif (row[0] == True) & (row[1] == True):
        return 'Both'
    else:
        return 'None'

# Map count_classification function to each row in y_set
results = pd.DataFrame(y_set)
results['observed'] = y_test
results['class'] = results.apply(count_classification, axis=1)
# Show first 5 instances
print(results[0:5])

In [ ]:
# Count the number of instances in each 'count'
results['class'].value_counts()
